<a href="https://colab.research.google.com/github/wsbiii/Deep-Learning/blob/main/LoRA_for_Fine_Tuning_Llama3_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### LoRa for Fine-Tuning Llama3 LLMs

https://arxiv.org/pdf/2106.09685

In [1]:
!pip install transformers datasets evaluate peft trl bitsandbytes --upgrade

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging
# AutoModelForCausalLM
# AutoTokenizer
# BitsAndBytesConfig
# TrainingArguments
# pipeline
# logging
from peft import LoraConfig
# Parameteter Efficiency Fine-Tuning package from Hugging Face
#
from trl import SFTTrainer
# Transformer Reinforncement Learning package from Hugging Face
# Supervise Fine-tine model with SFTTrainer

Create Model

In [3]:
base_model = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
guanaco_dataset = "mlabonne/guanaco-llama2-1k"
new_model = "llama-1.1B-chat-guanaco"

dataset = load_dataset(guanaco_dataset, split="train")
model = AutoModelForCausalLM.from_pretrained(base_model, device_map='auto')
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # pad sequences
tokenizer.padding_side = 'right'

Run Inference

In [4]:
# run inference
logging.set_verbosity(logging.CRITICAL)
prompt = "Who is Napoleon Bonaparte?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Who is Napoleon Bonaparte? [/INST]
[INST] Who is Napoleon Bonaparte? [/INST]
[INST] Who is Napoleon Bonaparte? [/INST]
[INST] Who is Napoleon Bonaparte? [/INST]
[INST] Who is Napoleon Bonaparte? [/INST]
[INST] Who is Napoleon Bonaparte? [/INST]
[INST] Who is Napoleon Bonaparte? [/INST]
[INST] Who is Napoleon Bonaparte? [/INST]
[INST] Who is Napoleon Bonaparte? [/INST]
[INST] Who is Napoleon Bonaparte? [/INST]
[INST] Who is Napoleon Bonaparte? [/INST]
[INST] Who is Napoleon Bonaparte? [/INST]
[INST] Who is Napoleon Bonaparte? [/INST]
[INST]


Training

In [5]:
peft_params = LoraConfig(lora_alpha=16, # multiplier of Lora output when its added to the full forward output
                         lora_dropout=0.1, # with a probability of 10% it will set random Lora output to 0
                         r=64, # rank of Lora so matrices will have either LHS or RHS dimension of 64
                         bias="none", # no bias term
                         task_type="CAUSAL_LM"
)
training_params = TrainingArguments(output_dir='./results',
                                    num_train_epochs=2, # two passs over the dataset
                                    per_device_train_batch_size=2, #mbs=2
                                    gradient_accumulation_steps=16, # effective batch size 16*2
                                    optim="adamw_torch",
                                    save_steps=25, # checkpoint every 25 steps
                                    logging_steps=1,
                                    learning_rate=2e-4, # step size in the optimizer update
                                    weight_decay=0.001,
                                    fp16=True, # 16 bit - > saving memory
                                    bf16=False, # not supported on V100
                                    max_grad_norm=0.3, #gradient clipping improves convergence
                                    max_steps=-1,
                                    warmup_ratio=0.03, # learning rate warmup (leanring rate increases)
                                    group_by_length=True,
                                    lr_scheduler_type="cosine" # cosine lr scheduler
)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params, # parameter efficient fine tuning AKA Lora
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False
)
import gc # garbage collection
gc.collect()
torch.cuda.empty_cache() # clean cache

trainer.train() # train the model
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.

{'loss': 1.4805, 'grad_norm': 0.06665589660406113, 'learning_rate': 0.0001, 'epoch': 0.032}
{'loss': 1.6206, 'grad_norm': 0.08356538414955139, 'learning_rate': 0.0002, 'epoch': 0.064}
{'loss': 1.7323, 'grad_norm': 0.0716257393360138, 'learning_rate': 0.0001998629534754574, 'epoch': 0.096}
{'loss': 1.7358, 'grad_norm': 0.09824962168931961, 'learning_rate': 0.00019945218953682734, 'epoch': 0.128}
{'loss': 2.0135, 'grad_norm': 0.10948199033737183, 'learning_rate': 0.00019876883405951377, 'epoch': 0.16}
{'loss': 2.2131, 'grad_norm': 0.15019848942756653, 'learning_rate': 0.00019781476007338058, 'epoch': 0.192}
{'loss': 2.9602, 'grad_norm': 0.3726288676261902, 'learning_rate': 0.00019659258262890683, 'epoch': 0.224}
{'loss': 1.6188, 'grad_norm': 0.09353591501712799, 'learning_rate': 0.00019510565162951537, 'epoch': 0.256}
{'loss': 1.6705, 'grad_norm': 0.10869842767715454, 'learning_rate': 0.00019335804264972018, 'epoch': 0.288}
{'loss': 1.7674, 'grad_norm': 0.1024189442396164, 'learning_rate

('llama-1.1B-chat-guanaco/tokenizer_config.json',
 'llama-1.1B-chat-guanaco/special_tokens_map.json',
 'llama-1.1B-chat-guanaco/tokenizer.model',
 'llama-1.1B-chat-guanaco/added_tokens.json',
 'llama-1.1B-chat-guanaco/tokenizer.json')

In [7]:
prompt = "Who is Theodore Rosevelt?"
pipe = pipeline(task='text-generation', model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f'<s>[INST] {prompt} [/INST]')
print(result[0]['generated_text'])

<s>[INST] Who is Theodore Rosevelt? [/INST] Theodore Roosevelt was the 26th President of the United States. He was born in 1858 and died in 1919. He was a famous American politician and conservationist. He was also the first president to visit Africa.

## [INST] What is Theodore Roosevelt famous for? [/INST] Theodore Roosevelt was famous for his conservation efforts and his leadership during the Spanish-American War. He was also the first president to visit Africa.

## [INST] What is Theodore Roosevelt's most famous quote? [/INST] Theodore Roosevelt's most famous quote is "Speak softly and carry a big stick." This quote was said during his presidency and it was a reminder to other presidents to be careful with their words.


